In [125]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import regex as re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import string
import shutil
import string


In [126]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,TextVectorization, GlobalAveragePooling1D

In [127]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [128]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [129]:
#Make vocabulary
def get_vocab(train,test):
    vocab = set(' '.join(test['text'].array.to_numpy()).split()).union(set(' '.join(train['text'].array.to_numpy()).split()))
    #Take out anything that doesn't have at least one letter or dash, or any links
    removed = {x for x in vocab if (not bool(re.match(r'^[a-zA-Z-]+[0-9]*$',x))) or x.startswith('http')}
    filtered = vocab - removed
    filtered = {f.strip(string.punctuation) for f in filtered}
    print(f'vocab size before filtering: {len(vocab)} \n vocab size after: {len(filtered)}')
    return filtered

def custom_preproccess(s):
    s = tf.strings.regex_replace(tf.strings.lower(s), '[^\w\s0-9]', '')
    s = tf.strings.regex_replace(s, '^https+$', '')
    return s

In [130]:
v = get_vocab(df_train,df_test)
vocab_size = len(v)
vocab_size

vocab size before filtering: 41747 
 vocab size after: 18557


18557

In [131]:
X = df_train['text'].array.to_numpy()
X.shape

(7613,)

In [132]:
Y = df_train['target'].array.to_numpy()
Y.shape

(7613,)

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=47)
X_train, X_val, y_train, y_val  = train_test_split(X_train,y_train,random_state=47)
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((4281,), (1904,), (1428,), (4281,), (1904,), (1428,))

Embedding Layer Test

In [134]:
embedding_layer = Embedding(vocab_size, 5)
embedding_layer(tf.constant([1, 2, 3])).numpy()

array([[-0.04235647,  0.00895909, -0.00239683,  0.02808202,  0.00782602],
       [ 0.0239282 , -0.01764734,  0.04864581, -0.04865059, -0.04713385],
       [-0.03522789,  0.01035714, -0.02550633, -0.03484509, -0.02608335]],
      dtype=float32)

In [135]:
embedding_dim= 4
sequence_length = 3

In [136]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [137]:
#embedding_layer = Embedding(vocab_size,embedding_dim, name="embedding")
#vectorize_layer = TextVectorization(max_tokens=vocab_size,
#    output_mode='int',
#    output_sequence_length=sequence_length)
#vectorize_layer.adapt(X)

In [138]:
vectorize_layer = TextVectorization(max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length,
    standardize=custom_preproccess)
vectorize_layer.adapt(X)

model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(vectorize_layer.vocabulary_size(),embedding_dim, name="embedding"))
model.add(GlobalAveragePooling1D())
model.add(Dense(embedding_dim, activation='relu'),)
model.add(Dense(1))

In [139]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [140]:
model.fit(X_train, y_train, epochs=10, batch_size=2000,callbacks=[tensorboard_callback])

Epoch 1/10
3/3 [==============================] - 1s 25ms/step - loss: 3.0864 - accuracy: 0.5707
Epoch 2/10
3/3 [==============================] - 0s 35ms/step - loss: 2.1814 - accuracy: 0.5707
Epoch 3/10
3/3 [==============================] - 0s 38ms/step - loss: 1.8759 - accuracy: 0.5707
Epoch 4/10
3/3 [==============================] - 0s 35ms/step - loss: 1.7204 - accuracy: 0.5707
Epoch 5/10
3/3 [==============================] - 0s 34ms/step - loss: 1.6227 - accuracy: 0.5707
Epoch 6/10
3/3 [==============================] - 0s 36ms/step - loss: 1.5511 - accuracy: 0.5707
Epoch 7/10
3/3 [==============================] - 0s 48ms/step - loss: 1.4931 - accuracy: 0.5707
Epoch 8/10
3/3 [==============================] - 0s 35ms/step - loss: 1.4451 - accuracy: 0.5707
Epoch 9/10
3/3 [==============================] - 0s 34ms/step - loss: 1.4035 - accuracy: 0.5707
Epoch 10/10
3/3 [==============================] - 0s 26ms/step - loss: 1.3669 - accuracy: 0.5707


In [142]:
model.evaluate(X_val,y_val)

45/45 [==============================] - 1s 3ms/step - loss: 1.4215 - accuracy: 0.5714


[1.4215328693389893, 0.5714285969734192]